In [1]:
# # this block of code imports graphical libraries for plotting graphs with high resolution
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
#import warnings
#warnings.filterwarnings('ignore')

In [2]:
# Libraries of functions need to be imported
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy import linalg
from scipy.interpolate import interp1d, LinearNDInterpolator, NearestNDInterpolator
from sklearn.decomposition import PCA

from scipy.spatial.distance import cdist

# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [4]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

In [5]:
# this is the correct vectorized version
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

In [6]:
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))

In [7]:
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

# Learning Objectives

-  How to efficiently compute all the pairwise distances among different observations.
- How to use the weights with the distances.
- Test the locally weighted regressors.
- Develop applications with boosting techniques.

In [8]:
# real data application includes cars.csv, concrete.csv and more
data = pd.read_csv('../Data Sets/cars.csv')

In [ ]:
data

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449
...,...,...,...,...
387,27.0,4,140.0,2790
388,44.0,4,97.0,2130
389,32.0,4,135.0,2295
390,28.0,4,120.0,2625


In [9]:
x = data.drop(columns=['MPG']).values
y = data['MPG'].values.reshape(-1,1)

## Compute all pairwise distances efficiently

This means we build a general case, assuming two matrices, we want to compute all the pairwise distances between the rows of matrix 1 and the rows of matrix 2.

The Euclidean distance between $\vec{u}$ and $\vec{v}$ is:

$$\large \text{dist}\left(\vec{u},\vec{v}\right):= \sqrt{\sum(u_i-v_i)^2}$$

In [11]:
# how about the case when v is a matrix with 5 columns?
def dist(u,v):
  D = []
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  # we would like all the pairwise combinations if u and v are matrices
  # we could avoid two for loops if we consider broadcasting
  for rowj in v:
    D.append(np.sqrt(np.sum((u-rowj)**2,axis=1)))
  return np.array(D).T

In [13]:
%time
dist(x,x)

CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 2.86 µs


array([[   0.        ,  193.82982227,   68.8839604 , ..., 1221.18016689,
         898.68014332,  806.23569755],
       [ 193.82982227,    0.        ,  258.98455552, ..., 1414.44158593,
        1092.4925629 , 1000.0529986 ],
       [  68.8839604 ,  258.98455552,    0.        , ..., 1155.58902729,
         834.8299228 ,  743.15072495],
       ...,
       [1221.18016689, 1414.44158593, 1155.58902729, ...,    0.        ,
         330.34073318,  425.30106983],
       [ 898.68014332, 1092.4925629 ,  834.8299228 , ...,  330.34073318,
           0.        ,   95.00526301],
       [ 806.23569755, 1000.0529986 ,  743.15072495, ...,  425.30106983,
          95.00526301,    0.        ]])

In [31]:
%time
cdist(x, x[0:2], metric='euclidean')

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


array([[   0.        ,  193.82982227],
       [ 193.82982227,    0.        ],
       [  68.8839604 ,  258.98455552],
       [  71.06335202,  264.03787607],
       [  55.22680509,  248.67649668],
       [ 845.84454837,  652.79782475],
       [ 862.61752822,  669.13152668],
       [ 818.87300603,  625.50859307],
       [ 932.81563023,  739.49239347],
       [ 355.81596367,  162.01543136],
       [  96.21330469,  134.12307781],
       [ 110.06361797,   84.59314393],
       [ 273.30934854,   84.40379138],
       [ 443.42755891,  616.01461022],
       [1148.51033953, 1342.09761195],
       [ 679.79849956,  873.33155216],
       [ 737.94850769,  931.32486276],
       [ 923.22369987, 1116.12723289],
       [1389.96115054, 1583.34898238],
       [1682.1643796 , 1875.15039397],
       [ 855.01403497, 1048.8360215 ],
       [1092.47059457, 1286.17028422],
       [1147.11202592, 1340.7669447 ],
       [1283.55443983, 1476.86763117],
       [ 862.78850247, 1055.8551037 ],
       [1112.263458  ,  9

In [10]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(cdist(u, v, metric='euclidean')/(2*tau))

## Scaling is very important for Locally Weighted Regression

In [16]:
# we show now the effect of scaling onto the data
scale = MinMaxScaler()

In [29]:
# Example of the weights matrix if we used all the data
W = weight_function(scale.fit_transform(x),scale.fit_transform(x),kern=Tricubic,tau=0.3)

In [30]:
W

array([[1.        , 0.97415419, 0.9994374 , ..., 0.        , 0.        ,
        0.        ],
       [0.97415419, 1.        , 0.98152172, ..., 0.        , 0.        ,
        0.        ],
       [0.9994374 , 0.98152172, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.98564254,
        0.97155841],
       [0.        , 0.        , 0.        , ..., 0.98564254, 1.        ,
        0.99972486],
       [0.        , 0.        , 0.        , ..., 0.97155841, 0.99972486,
        1.        ]])

# The Lowess Class - Version without Triangulation

In [66]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:

          n = len(x_new)
          yest_test = []
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test.append(lm.predict(x_new[i].reshape(1,-1)))
        return np.array(yest_test).reshape(-1,1)

In [67]:
model = Lowess(kernel=Epanechnikov,tau=0.05)

In [68]:
xscaled = scale.fit_transform(x)

In [69]:
model.fit(xscaled,y)

In [70]:
yhat = model.predict(xscaled)

In [71]:
yhat

array([[17.31906616],
       [16.27981387],
       [16.78239778],
       [17.29858228],
       [17.38272896],
       [14.38184159],
       [14.08272459],
       [14.20440129],
       [14.0976539 ],
       [15.13812436],
       [14.84611049],
       [15.65519283],
       [15.06666448],
       [13.98855584],
       [27.53897403],
       [21.28944226],
       [21.151905  ],
       [21.17873764],
       [31.48045272],
       [32.48030196],
       [25.7717839 ],
       [27.74608895],
       [28.1956697 ],
       [27.98596705],
       [21.1860469 ],
       [12.59811335],
       [13.51335025],
       [13.74531527],
       [11.26814365],
       [31.48045272],
       [26.73761298],
       [28.67637648],
       [19.09643053],
       [19.26209254],
       [17.93023186],
       [18.04365862],
       [19.36185146],
       [14.50738194],
       [14.13195418],
       [14.6910138 ],
       [14.89550395],
       [11.8675787 ],
       [13.4209999 ],
       [12.45499047],
       [24.32407556],
       [26

In [72]:
mse(yhat,y)

13.045495055465093

In [74]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_lw = Lowess(kernel= Epanechnikov,tau=0.14)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 16.455751824171323
The Cross-validated Mean Squared Error for Random Forest is : 17.605429519832445


# Boosting Applications